In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,StandardScaler
from sklearn.metrics import accuracy_score


1. Load datasets and preprocessing

In [16]:


data = pd.read_csv('Dataset/csgo.csv')


data = data.drop(['day', 'month', 'year', 'date',  
                'team_a_rounds', 'team_b_rounds'], axis=1)


onehot_encoder = OneHotEncoder()
map_encoded = onehot_encoder.fit_transform(data[["map"]]).toarray()  

label_encoder = LabelEncoder()
data["result"] = label_encoder.fit_transform(data["result"])

map_encoded_data = pd.DataFrame(map_encoded, columns=onehot_encoder.get_feature_names_out(["map"]))


data = pd.concat([map_encoded_data, data.drop(columns=["map"])], axis=1)

scaler = StandardScaler()
x = data.drop('result', axis=1)
x = scaler.fit_transform(x)
y = data['result']
y = y.values.reshape(-1,)
print(y)




[2 0 2 ... 0 0 2]


2. Split datasets into training and test sets

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)


print(x_train.dtype)

float64


3. Using Multi Layer Perceptron to predict if new datasets, it will be lose or win or tie, we will try with only one hidden layer 

In [18]:
def softmax(V):
    e_v = np.exp(V - np.max(V, axis=1, keepdims=True))
    return e_v / e_v.sum(axis=1, keepdims=True)


def convert_labels(y, C=3):
    Y = np.zeros((len(y), C))  
    Y[np.arange(len(y)), y] = 1
    return Y


def cost(Y, Ypred):
    return -np.sum(Y * np.log(Ypred + 1e-8)) / Y.shape[0]  


dim1 = x.shape[1]  
unitLayer = 100
dim2 = C = 3  
N = x_train.shape[0]  

Y = convert_labels(y_train, C) 


W1 = 0.01 * np.random.randn(dim1, unitLayer)
b1 = np.zeros((1, unitLayer))
W2 = 0.01 * np.random.randn(unitLayer, dim2)
b2 = np.zeros((1, dim2))

learningRate = 1


for i in range(10000):

    # FeedForward

    Z1 = np.dot(x_train, W1) + b1 
    A1 = np.maximum(Z1, 0)  
    Z2 = np.dot(A1, W2) + b2  
    Ypred = softmax(Z2) 

    if i % 1000 == 0:
        print(f"Iteration {i}, Cost: {cost(Y, Ypred)}")

    # Backpropagation
    E2 = (Ypred - Y) / N  
    dW2 = np.dot(A1.T, E2)  
    db2 = np.sum(E2, axis=0, keepdims=True)  

    E1 = np.dot(E2, W2.T)  
    E1[Z1 <= 0] = 0  

    dW1 = np.dot(x_train.T, E1)  
    db1 = np.sum(E1, axis=0, keepdims=True)  
    #Update weights and biases
    W1 -= learningRate * dW1
    b1 -= learningRate * db1
    W2 -= learningRate * dW2
    b2 -= learningRate * db2



Iteration 0, Cost: 1.0984862623534384
Iteration 1000, Cost: 0.2413137475285645
Iteration 2000, Cost: 0.1659694498177169
Iteration 3000, Cost: 0.016147032122156794
Iteration 4000, Cost: 0.007215102114106235
Iteration 5000, Cost: 0.004436451348836866
Iteration 6000, Cost: 0.003124613634228786
Iteration 7000, Cost: 0.0023773234029422627
Iteration 8000, Cost: 0.0019006218111765035
Iteration 9000, Cost: 0.0015713275108854535


In [19]:

Z1_test = np.dot(x_test, W1) + b1  
A1_test = np.maximum(Z1_test, 0)  
Z2_test = np.dot(A1_test, W2) + b2  
Ypred_test = softmax(Z2_test)  


y_pred_labels = np.argmax(Ypred_test, axis=1)  


accuracy = np.mean(y_pred_labels == y_test) * 100
print(f"🎯 Accuracy on test set: {accuracy:.2f}%")


🎯 Accuracy on test set: 68.28%


In [20]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


data = pd.read_csv('Dataset/csgo.csv')


data = data.drop(['day', 'month', 'year', 'date', 'team_a_rounds', 'team_b_rounds'], axis=1)


onehot_encoder = OneHotEncoder()
map_encoded = onehot_encoder.fit_transform(data[["map"]]).toarray()
map_encoded_data = pd.DataFrame(map_encoded, columns=onehot_encoder.get_feature_names_out(["map"]))


label_encoder = LabelEncoder()
data["result"] = label_encoder.fit_transform(data["result"])


data = pd.concat([map_encoded_data, data.drop(columns=["map"])], axis=1)

scaler = StandardScaler()
x = data.drop('result', axis=1)
x = scaler.fit_transform(x)
y = data['result'].values.reshape(-1,)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


mlp = MLPClassifier(hidden_layer_sizes=(128, 64, 32, 18), 
                    activation='relu',         
                    solver='adam',             
                    batch_size=32,             
                    learning_rate_init=0.01,   
                    max_iter=10000,            
                    random_state=42)


mlp.fit(x_train, y_train)


y_pred = mlp.predict(x_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy:.4f}")


Accuracy on test set: 0.7004
